<a href="https://colab.research.google.com/github/forminju/youtuber-look-alike-crawler/blob/main/arcface_look_alike.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 모델 학습을 위한 전처리 과정

In [1]:
# 토치 버전확인
import torch
print(torch.__version__)

1.13.1+cu116


In [2]:
# 라이브러리 세팅

import random
import pandas as pd
import numpy as np
import os
import cv2
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import albumentations as A #증강
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings(action='ignore')

from sklearn.model_selection import train_test_split

In [3]:
# GPU 확인
print(torch.cuda.is_available())
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

True


In [4]:
# EfficientNet 오류 안나게..
torch.backends.cudnn.enabled = False

In [5]:
CFG = {
    'IMG_SIZE':456,
    'EPOCHS':10,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':16,
    'SEED':42
}

# chatGPT 추천
# CFG = {
#     'embedding_size': 512,
#     'num_classes': num_classes,
#     'margin': 0.5,
#     'scale': 64,
#     'batch_size': 32,
#     'learning_rate': 0.0001,
#     'num_epochs': 50
# }

In [6]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED'])

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
df = pd.read_csv('/content/drive/MyDrive/glory_match/look-alike/glory_train_df.csv')
image_root_dir = '/content/drive/MyDrive/glory_match/look-alike/pre-processed-image'

### Train, Test dataset 분리
항목별로 70%를 train, 30%를 valid로 사용합니다.

In [9]:
class_name_list = []

In [10]:
tmp=df.copy()
tmp['class'].shape
for initial in tmp['class']:
  if initial not in class_name_list:
    class_name_list.append(initial)
class_name_list

['SMO', 'CHJ', 'CDE', 'HAR', 'JJJ', 'JSI', 'OJY', 'SHE', 'SHG', 'IDH', 'IJH']

In [11]:
class_name_to_lower_case_list = [class_name.lower() for class_name in class_name_list]
class_name_to_lower_case_list

['smo', 'chj', 'cde', 'har', 'jjj', 'jsi', 'ojy', 'she', 'shg', 'idh', 'ijh']

In [12]:
for initial in class_name_to_lower_case_list:
  image_root_path = os.path.join(image_root_dir, initial)
  for i in range(len(tmp)):
    if (initial.upper() == tmp['class'][i]):
      tmp['file_name'][i] = os.path.join(image_root_path, tmp['file_name'][i])
tmp['file_name'].head()

0    /content/drive/MyDrive/glory_match/look-alike/...
1    /content/drive/MyDrive/glory_match/look-alike/...
2    /content/drive/MyDrive/glory_match/look-alike/...
3    /content/drive/MyDrive/glory_match/look-alike/...
4    /content/drive/MyDrive/glory_match/look-alike/...
Name: file_name, dtype: object

In [13]:
train = pd.DataFrame(columns=tmp.columns)
valid = pd.DataFrame(columns=tmp.columns)

for class_name in class_name_list:
  tmp_with_class = tmp.loc[tmp['class'] == class_name]
  train_tmp, valid_tmp = train_test_split(tmp_with_class, test_size = 0.3, random_state = 42)

  train_tmp['class'] = class_name
  valid_tmp['class'] = class_name
  train = pd.concat([train, train_tmp])
  valid = pd.concat([valid, valid_tmp])

In [14]:
train

,index,file_name,class
41,41,/content/drive/MyDrive/glory_match/look-alike/...,SMO
19,19,/content/drive/MyDrive/glory_match/look-alike/...,SMO
30,30,/content/drive/MyDrive/glory_match/look-alike/...,SMO
49,49,/content/drive/MyDrive/glory_match/look-alike/...,SMO
50,50,/content/drive/MyDrive/glory_match/look-alike/...,SMO
...,...,...,...
1213,1213,/content/drive/MyDrive/glory_match/look-alike/...,IJH
1121,1121,/content/drive/MyDrive/glory_match/look-alike/...,IJH
1199,1199,/content/drive/MyDrive/glory_match/look-alike/...,IJH
1286,1286,/content/drive/MyDrive/glory_match/look-alike/...,IJH


### label을 tensor로 반환하기 위해 one-hot-encoding 적용

In [15]:
one_hot_encoded = pd.get_dummies(tmp['class'])
train_one_hot_encoded = pd.get_dummies(train['class'])
valid_one_hot_encoded = pd.get_dummies(valid['class'])

In [16]:
data = pd.concat([tmp, one_hot_encoded], axis=1)
data = data.drop(['class'], axis=1)
train = pd.concat([train, train_one_hot_encoded], axis=1)
train = train.drop(['class'], axis=1)
valid = pd.concat([valid, valid_one_hot_encoded], axis=1)
valid = valid.drop(['class'], axis=1)

In [17]:
train.head()

,index,file_name,CDE,CHJ,HAR,IDH,IJH,JJJ,JSI,OJY,SHE,SHG,SMO
41,41,/content/drive/MyDrive/glory_match/look-alike/...,0,0,0,0,0,0,0,0,0,0,1
19,19,/content/drive/MyDrive/glory_match/look-alike/...,0,0,0,0,0,0,0,0,0,0,1
30,30,/content/drive/MyDrive/glory_match/look-alike/...,0,0,0,0,0,0,0,0,0,0,1
49,49,/content/drive/MyDrive/glory_match/look-alike/...,0,0,0,0,0,0,0,0,0,0,1
50,50,/content/drive/MyDrive/glory_match/look-alike/...,0,0,0,0,0,0,0,0,0,0,1


In [18]:
valid.head()

,index,file_name,CDE,CHJ,HAR,IDH,IJH,JJJ,JSI,OJY,SHE,SHG,SMO
0,0,/content/drive/MyDrive/glory_match/look-alike/...,0,0,0,0,0,0,0,0,0,0,1
5,5,/content/drive/MyDrive/glory_match/look-alike/...,0,0,0,0,0,0,0,0,0,0,1
34,34,/content/drive/MyDrive/glory_match/look-alike/...,0,0,0,0,0,0,0,0,0,0,1
13,13,/content/drive/MyDrive/glory_match/look-alike/...,0,0,0,0,0,0,0,0,0,0,1
44,44,/content/drive/MyDrive/glory_match/look-alike/...,0,0,0,0,0,0,0,0,0,0,1


# CustomDataset

In [19]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = cv2.imread(img_path)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = torch.FloatTensor(self.label_list[index])
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [20]:
train_transform = A.Compose([
    A.Resize(180, 180),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.RandomBrightnessContrast(brightness_limit=(-0.3, 0.3), contrast_limit=(-0.3, 0.3), p=1),
    A.ChannelShuffle(p=0.2),
    ToTensorV2()
])

valid_transform = A.Compose([
    A.Resize(180, 180),
    ToTensorV2()
])

In [21]:
def get_labels(df):
    return df.iloc[:,2:].values

In [22]:
train_labels=get_labels(train)
valid_labels=get_labels(valid)

In [23]:
train_dataset = CustomDataset(train['file_name'].values, train_labels, transforms=train_transform)
valid_dataset = CustomDataset(valid['file_name'].values, valid_labels, transforms=valid_transform)

train_loader = DataLoader(train_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True, num_workers=4)
valid_loader = DataLoader(valid_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=4)

In [24]:
for x,y in train_loader:
  print(f'Image Shape: {x.shape}')
  print(f'Label Shape: {y.shape}')
  break

Image Shape: torch.Size([16, 3, 180, 180])
Label Shape: torch.Size([16, 11])


# ArcFace

In [38]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import math

class ArcMarginProduct(nn.Module):
    def __init__(self, in_features, out_features, scale=30.0, margin=0.50, easy_margin=False, device='cuda'):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.scale = scale
        self.margin = margin
        self.device = device
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(margin)
        self.sin_m = math.sin(margin)
        self.th = math.cos(math.pi - margin)
        self.mm = math.sin(math.pi - margin) * margin

    def forward(self, input, label):
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        sine = torch.sqrt(1.0 - torch.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)

        one_hot = torch.zeros(cosine.size(), device=input.device)
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.scale

        return output

class CustomArcFaceModel(nn.Module):
    def __init__(self, num_classes=11, device='cuda'):
        super(CustomArcFaceModel, self).__init__()
        self.device = device
        self.backbone = nn.Sequential(*list(models.resnet50(pretrained=True).children())[:-1])
        self.arc_margin_product = ArcMarginProduct(2048, num_classes, device=self.device)
        self.classifier = nn.Linear(2048, num_classes)

    def forward(self, x):
        features = self.backbone(x)
        features = F.normalize(features)
        features = features.view(features.size(0), -1)
        features = F.sigmoid(self.classifier(features))

        return features        


In [39]:
def train(model, optimizer, train_loader, valid_loader, scheduler, device):
    model.to(device)
    criterion = nn.BCELoss().to(device)
    
    best_valid_acc = 0
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _valid_loss, _valid_acc = validation(model, criterion, valid_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Valid Loss : [{_valid_loss:.5f}] Valid ACC : [{_valid_acc:.5f}]')
        
        if scheduler is not None:
            scheduler.step(_valid_acc)
            
        if best_valid_acc < _valid_acc:
            best_valid_acc = _valid_acc
            best_model = model
    
    return best_model

In [40]:
def validation(model, criterion, valid_loader, device):
    model.eval()
    valid_loss = []
    valid_acc = []
    with torch.no_grad():
        for imgs, labels in tqdm(iter(valid_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            probs = model(imgs)
            
            loss = criterion(probs, labels)
            
            probs  = probs.cpu().detach().numpy()
            labels = labels.cpu().detach().numpy()
            preds = probs > 0.5
            batch_acc = (labels == preds).mean()
            
            valid_acc.append(batch_acc)
            valid_loss.append(loss.item())
        
        _valid_loss = np.mean(valid_loss)
        _valid_acc = np.mean(valid_acc)
    
    return _valid_loss, _valid_acc

In [41]:
model = CustomArcFaceModel()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2,threshold_mode='abs',min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, valid_loader, scheduler, device)

  0%|          | 0/57 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

Epoch [1], Train Loss : [0.56055] Valid Loss : [0.45991] Valid ACC : [0.90909]


  0%|          | 0/57 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

...

  0%|          | 0/25 [00:00<?, ?it/s]

Epoch [9], Train Loss : [0.25939] Valid Loss : [0.26170] Valid ACC : [0.90909]


  0%|          | 0/57 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

Epoch [10], Train Loss : [0.25280] Valid Loss : [0.25341] Valid ACC : [0.90909]
Epoch 00010: reducing learning rate of group 0 to 3.7500e-05.


In [42]:
PATH = '/content/drive/MyDrive/glory_match/'

In [43]:
torch.save(model, PATH + 'arcface.pt') 

In [44]:
torch.save(model.state_dict(), PATH + 'arcface_dict.pt')

In [45]:
torch.save({
    'arcface': model.state_dict(),
    'optimizer': optimizer.state_dict()
}, PATH + 'arcface_all.tar') 

# ResNet

In [ ]:
class ResNet50(nn.Module):
    def __init__(self, num_classes=11):
        super(ResNet50, self).__init__()
        self.backbone = models.resnet18(pretrained=True)
        self.classifier = nn.Linear(1000, num_classes)
        
        ######
        
    def forward(self, x):
        x = self.backbone(x)
        
        ######
        
        x = F.sigmoid(self.classifier(x))
        return x

In [ ]:
def train(model, optimizer, train_loader, valid_loader, scheduler, device):
    model.to(device)
    criterion = nn.BCELoss().to(device)
    
    best_valid_acc = 0
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _valid_loss, _valid_acc = validation(model, criterion, valid_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Valid Loss : [{_valid_loss:.5f}] Valid ACC : [{_valid_acc:.5f}]')
        
        if scheduler is not None:
            scheduler.step(_valid_acc)
            
        if best_valid_acc < _valid_acc:
            best_valid_acc = _valid_acc
            best_model = model
    
    return best_model

In [ ]:
def validation(model, criterion, valid_loader, device):
    model.eval()
    valid_loss = []
    valid_acc = []
    with torch.no_grad():
        for imgs, labels in tqdm(iter(valid_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            probs = model(imgs)
            
            loss = criterion(probs, labels)
            
            probs  = probs.cpu().detach().numpy()
            labels = labels.cpu().detach().numpy()
            preds = probs > 0.5
            batch_acc = (labels == preds).mean()
            
            valid_acc.append(batch_acc)
            valid_loss.append(loss.item())
        
        _valid_loss = np.mean(valid_loss)
        _valid_acc = np.mean(valid_acc)
    
    return _valid_loss, _valid_acc

In [ ]:
model = ResNet50()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2,threshold_mode='abs',min_lr=1e-8, verbose=True)
infer_model = train(model, optimizer, train_loader, valid_loader, scheduler, device)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

  0%|          | 0/57 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

Epoch [1], Train Loss : [0.26212] Valid Loss : [0.18140] Valid ACC : [0.93000]


...

  0%|          | 0/25 [00:00<?, ?it/s]

Epoch [9], Train Loss : [0.03098] Valid Loss : [0.11877] Valid ACC : [0.96490]


  0%|          | 0/57 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

Epoch [10], Train Loss : [0.02809] Valid Loss : [0.09940] Valid ACC : [0.97244]


In [ ]:
PATH = '/content/drive/MyDrive/glory_match/'

In [ ]:
torch.save(model, PATH + 'resnet18.pt') 
torch.save(model.state_dict(), PATH + 'resnet18_dict.pt')

torch.save({
    'resnet18': model.state_dict(),
    'optimizer': optimizer.state_dict()
}, PATH + 'resnet18_all.tar') 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import math

class ArcMarginProduct(nn.Module):
    def __init__(self, in_features, out_features, scale=30.0, margin=0.50, easy_margin=False, device='cuda'):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.scale = scale
        self.margin = margin
        self.device = device
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(margin)
        self.sin_m = math.sin(margin)
        self.th = math.cos(math.pi - margin)
        self.mm = math.sin(math.pi - margin) * margin

    def forward(self, input, label):
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        sine = torch.sqrt(1.0 - torch.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)

        one_hot = torch.zeros(cosine.size(), device=input.device)
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.scale

        return output

class CustomArcFaceModel(nn.Module):
    def __init__(self, num_classes, device='cuda'):
        super(CustomArcFaceModel, self).__init__()
        self.device = device
        self.backbone = nn.Sequential(*list(models.resnet50(pretrained=True).children())[:-1])
        self.arc_margin_product = ArcMarginProduct(2048, num_classes, device=self.device)
        self.classifier = nn.Linear(1000, num_classes)

    def forward(self, x, labels=None):
        features = self.backbone(x)
        features = F.normalize(features)
        features = features.view(features.size(0), -1)
        features = F.sigmoid(self.classifier)
        if labels is not None:
            logits = self.arc_margin_product(features, labels)
            return logits
        return features

In [ ]:
num_classes = 11  # 분류할 클래스의 수 (CDE, CHJ, HAR, IDH, IJH, JJJ, JSI, OJY, SHE, SHG, SMO)
embedding_size = 2048
learning_rate = 0.001
epochs = 10

# 모델 생성
model = CustomArcFaceModel(num_classes, embedding_size)

# 옵티마이저 및 손실 함수 설정
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# 모델 학습
trained_model = train(model, optimizer, criterion, train_loader, valid_loader, device, epochs)


TypeError: ignored

In [ ]:
model = CustomArcFaceModel()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2,threshold_mode='abs',min_lr=1e-8, verbose=True)
infer_model = train(model, optimizer, train_loader, valid_loader, scheduler, device)

TypeError: ignored

# TestAccuracy 확인

In [ ]:
# Function to test the model with the test dataset and print the accuracy for the test images
def testAccuracy():
    
    model.eval()
    accuracy = 0.0
    total = 0.0
    
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            # run the model on the test set to predict labels
            outputs = model(images)
            # the label with the highest energy will be our prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            accuracy += (predicted == labels).sum().item()
    
    # compute the accuracy over all test images
    accuracy = (100 * accuracy / total)
    return(accuracy)